In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, warnings, scipy 
#import plotly.offline as py
#import plotly.figure_factory as ff
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#from plotly import tools
#from plotly.subplots import make_subplots
from pyspark.sql import SparkSession
from pyspark.sql import Row
import numpy as np
import pandas as pd
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import pyspark.sql.functions as fn
from pyspark.sql.functions import exp, corr
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import isnan, when, count, col, isnull

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
#DirectoryPath="D:\\BDAPROJECT\\data\\"  #Lakshya Directory path
#DirectoryPath=""  #Sankalp Directory path
DirectoryPath="/FileStore/tables"  #Abhiraj Directory path
#DirectoryPath=""  #Surjit Directory path

In [4]:
flightPath=DirectoryPath+"flights.csv"
airportPath=DirectoryPath+"airports.csv"
airlinesPath=DirectoryPath+"airlines.csv"

In [5]:
airportPath = spark.read.option('delimiter' , ',').csv(get_training_filename('airports.csv'),inferSchema = True, header=True)
flightPath = spark.read.option('delimiter' , ',').csv(get_training_filename('flights.csv'),inferSchema = True, header=True)
airlinesPath = spark.read.option('delimiter' , ',').csv(get_training_filename('airlines.csv'),inferSchema = True, header=True)


In [6]:
print(flightPath.count(),len(flightPath.columns))

5819079 31

In [7]:
flightPath.columns

Out[7]: ['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY']

In [8]:
#CHECK FOR THE NULL VALUES IN THE COLUMNS
flightPath.select([count(when(isnull(c), c)).alias(c) for c in flightPath.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
 0| 0| 0| 0| 0| 0| 14721| 0| 0| 0| 86153| 86153| 89047| 89047| 6| 105071| 105071| 0| 92513| 92513| 0| 92513| 105071| 0| 0| 5729195| 4755640| 4755640| 4755640| 4755640| 4755640|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+

In [9]:
flightPath.printSchema

Out[54]: <bound method DataFrame.printSchema of DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int]>

In [10]:
#SHOWS COUNT OF DISTINCT AIRLINES WITH THEIR ABBERIVIATIONS
flightPath.select('AIRLINE').distinct().show()
flightPath.select('AIRLINE').distinct().count()

+-------+
AIRLINE|
+-------+
 UA|
 NK|
 AA|
 EV|
 B6|
 DL|
 OO|
 F9|
 US|
 MQ|
 HA|
 AS|
 VX|
 WN|
+-------+

Out[57]: 14

In [11]:
#SHOWS THE TOTAL/MOST CANCELATIONS BY INDIVIDUAL AIRLINES

df = flightPath.groupBy('AIRLINE').sum('CANCELLED').toPandas()
df
ax = sns.barplot(x = 'AIRLINE', y = 'sum(CANCELLED)', data = df)
display()

In [12]:
#SHOWS THE AIRLINE WITH MAXIMUM/MINIMUM DELAYS

df1 = flightPath.groupBy('AIRLINE').sum('AIRLINE_DELAY').toPandas()
df1
fig2 = sns.barplot(x = 'AIRLINE', y = 'sum(AIRLINE_DELAY)', data = df1)
display()

In [13]:
#SHOWS WHICH DAY HAS MAXIMUM/MINIMUM DELAY

df2 = flightPath.groupBy('DAY_OF_WEEK').avg('AIRLINE_DELAY').toPandas()
df2
fig3 = sns.barplot(x = 'DAY_OF_WEEK', y = 'avg(AIRLINE_DELAY)', data = df2)
display()

In [14]:
#SHOWS WHICH MONTH HAS MAXIMUM/MINIMUM DELAY

df3 = flightPath.groupBy('MONTH').avg('AIRLINE_DELAY').toPandas()
df3
fig4 = sns.barplot(x = 'MONTH', y = 'avg(AIRLINE_DELAY)', data = df3)
display()


In [15]:
df5 =  flightPath.groupBy('AIRLINE').avg('LATE_AIRCRAFT_DELAY')
df5 = df5.toPandas()
fig4 = sns.barplot(x = 'AIRLINE', y = 'avg(LATE_AIRCRAFT_DELAY)', data = df5)
display()

In [16]:
#dataframe to evaluate factors that might affect the cancelations. 
flights_cancel = flightPath.filter(flightPath.CANCELLED == 1)
flights_cancel.groupBy('AIRLINE').sum('CANCELLED').show()

+-------+--------------+
AIRLINE|sum(CANCELLED)|
+-------+--------------+
 UA| 6573|
 NK| 2004|
 AA| 10919|
 EV| 15231|
 B6| 4276|
 DL| 3824|
 OO| 9960|
 F9| 588|
 US| 4067|
 MQ| 15025|
 HA| 171|
 AS| 669|
 VX| 534|
 WN| 16043|
+-------+--------------+

In [17]:
#CHECK FOR THE NULL VALUES IN THE COLUMNS of FLIGHT_CANCEL
flights_cancel.select([count(when(isnull(c), c)).alias(c) for c in flights_cancel.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
 0| 0| 0| 0| 0| 0| 14721| 0| 0| 0| 86153| 86153| 89047| 89047| 5| 89884| 89884| 0| 89884| 89884| 0| 89884| 89884| 0| 0| 0| 89884| 89884| 89884| 89884| 89884|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+

In [18]:
#merge YEAR, Month, day into one column DATE
from pyspark.sql import functions as fn
flightPath = flightPath.withColumn("DATE",fn.concat_ws(' ', flightPath.YEAR,flightPath.MONTH,flightPath.DAY))
flightPath.select('DATE').show()

#REMOVE DAY, DATE, MONTH FROM FLIGHTPATH DATAFRAME

flightPath =  flightPath.drop('YEAR','MONTH','DAY')
flightPath.show()

+--------+
 DATE|
+--------+
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
2015 1 1|
+--------+
only showing top 20 rows

+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------+
DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY| DATE|
+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+--------+
 4| AS| 98| N407AS| ANC| SEA| 5| 2354| -11| 21| 15| 205| 194| 169| 1448| 404| 4| 430| 408| -22| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| AA| 2336| N3KUAA| LAX| PBI| 10| 2| -8| 12| 14| 280| 279| 263| 2330| 737| 4| 750| 741| -9| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| US| 840| N171US| SFO| CLT| 20| 18| -2| 16| 34| 286| 293| 266| 2296| 800| 11| 806| 811| 5| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| AA| 258| N3HYAA| LAX| MIA| 20| 15| -5| 15| 30| 285| 281| 258| 2342| 748| 8| 805| 756| -9| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| AS| 135| N527AS| SEA| ANC| 25| 24| -1| 11| 35| 235| 215| 199| 1448| 254| 5| 320| 259| -21| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| DL| 806| N3730B| SFO| MSP| 25| 20| -5| 18| 38| 217| 230| 206| 1589| 604| 6| 602| 610| 8| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| NK| 612| N635NK| LAS| MSP| 25| 19| -6| 11| 30| 181| 170| 154| 1299| 504| 5| 526| 509| -17| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| US| 2013| N584UW| LAX| CLT| 30| 44| 14| 13| 57| 273| 249| 228| 2125| 745| 8| 803| 753| -10| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| AA| 1112| N3LAAA| SFO| DFW| 30| 19| -11| 17| 36| 195| 193| 173| 1464| 529| 3| 545| 532| -13| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| DL| 1173| N826DN| LAS| ATL| 30| 33| 3| 12| 45| 221| 203| 186| 1747| 651| 5| 711| 656| -15| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| DL| 2336| N958DN| DEN| ATL| 30| 24| -6| 12| 36| 173| 149| 133| 1199| 449| 4| 523| 453| -30| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| AA| 1674| N853AA| LAS| MIA| 35| 27| -8| 21| 48| 268| 266| 238| 2174| 746| 7| 803| 753| -10| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| DL| 1434| N547US| LAX| MSP| 35| 35| 0| 18| 53| 214| 210| 188| 1535| 601| 4| 609| 605| -4| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| DL| 2324| N3751B| SLC| ATL| 40| 34| -6| 18| 52| 215| 199| 176| 1590| 548| 5| 615| 553| -22| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| DL| 2440| N651DL| SEA| MSP| 40| 39| -1| 28| 107| 189| 198| 166| 1399| 553| 4| 549| 557| 8| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| AS| 108| N309AS| ANC| SEA| 45| 41| -4| 17| 58| 204| 194| 173| 1448| 451| 4| 509| 455| -14| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| DL| 1560| N3743H| ANC| SEA| 45| 31| -14| 25| 56| 210| 200| 171| 1448| 447| 4| 515| 451| -24| 0| 0| null| null| null| null| null| null|2015 1 1|
 4| UA| 1197| N78448| SFO| IAH| 48| 42| -6| 11| 53| 218| 217| 199| 1635| 612| 7|

In [19]:
#drop the columns with more than 90% NA values
# lst = ['CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']

# for i in lst:
#   flightPath= flightPath.drop(i)
# flightPath.columns  

# #drop other columns which we might not take under consideration

# lst2 = ['TAXI_OUT','WHEELS_OFF','TAXI_IN','WHEELS_ON']
# for j in lst2:
#   flightPath= flightPath.drop(j)
# flightPath.columns  